In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

import xgboost
from xgboost import XGBRegressor
!pip install lightgbm
from lightgbm import LGBMRegressor
from lightgbm import plot_importance


# Model Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


##  DATA UNDERSTANDING

**Output**:

**Salary:** Annual salaries of the players in thousands of dollars

**Features**:

for the season 1986 - 1987:

* AtBat: Number of times at bat 
* Hits: Number of hits in
* HmRun: Number of home runs 
* Runs: Number of runs 
* RBI: Number of runs batted
* Walks: Number of walks 
* Years: Number of years in the major leagues
* CAtBat: Number of times at bat during his career
* CHits: Number of hits during his career
* CHmRun: Number of home runs during his career
* CRuns: Number of runs during his career
* CRBI: Number of runs batted in during his career
* CWalks: Number of walks during his career
* League: A factor with levels A and N indicating player’s league at the end of 1986
* Division: A factor with levels E and W indicating player’s division at the end of 1986
* NewLeague: A factor with levels A and N indicating player’s league at the beginning of 1987
* PutOuts: Number of put outs
* Assists: Number of assists
* Errors: Number of errors

**GOAL :** 

* PREDICT THE SALARY, WITH THE ALGORITHM WHICH GIVES THE MINIMUM ERROR. 

In [ ]:
hitters=pd.read_csv("../input/hitters/Hitters.csv")

In [ ]:
df = hitters.copy()

In [ ]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Decribe #####################")
    print(dataframe.describe().T)
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.70, 0.95, 0.99, 1]).T)
    
    
check_df(df)

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):

    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    # cat_cols + num_cols + cat_but_car = değişken sayısı.
    # num_but_cat cat_cols'un içerisinde zaten.
    # dolayısıyla tüm şu 3 liste ile tüm değişkenler seçilmiş olacaktır: cat_cols + num_cols + cat_but_car
    # num_but_cat sadece raporlama için verilmiştir.

    return cat_cols, cat_but_car, num_cols, num_but_cat

In [ ]:
grab_col_names(df)

In [ ]:
cat_cols, cat_but_car, num_cols, num_but_cat = grab_col_names(df)


## EDA

In [ ]:
plt.figure(figsize=(10,10))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(df.corr(), cmap=cmap, vmin=-1, vmax=1,  cbar_kws={"shrink": .8}, square=True, annot=True, fmt='.2f',linewidths=.8)

plt.show()

![](http://miro.medium.com/max/510/1*hxVvqttoCSkUT2_R1zA0Tg.gif)

**Positively skewed data:**
* If tail is on the right as that of the second image in the figure, it is right skewed data. It is also called positive skewed data.
* Common transformations of this data include square root, cube root, and log.

In [ ]:
def column_dplots():
    
    # 'CRBI','CRuns', 'CHits','CAtBat', 'CWalks'

    f,ax=plt.subplots(1,5,figsize=(27,7))
    
    ax[0].set_xlabel('CRBI',fontsize=20)
    sns.distplot(df.CRBI, ax = ax[0], color= "orange")
    ax[1].set_xlabel('CRuns',fontsize=20)
    sns.distplot(df.CRuns,ax=ax[1])
    ax[2].set_xlabel('CHits',fontsize=20)
    sns.distplot(df.CHits, ax=ax[2],color= "purple")
    ax[3].set_xlabel('CAtBat',fontsize=20)
    sns.distplot(df.CAtBat, ax=ax[3],color= "green" )
    ax[4].set_xlabel('CWalks',fontsize=20)
    sns.distplot(df.CWalks, ax=ax[4], color= "pink")


    plt.show()

column_dplots()

In [ ]:

def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))

    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()
        

for i in cat_cols:
    cat_summary(df,i,plot=True)

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()
        

for i in num_cols:
    num_summary(df,i,plot=True)

In [ ]:
sns.distplot(df.Salary)


### FEATURE GENERATION

In [ ]:
df.corr()

In [ ]:
df["AVG_HITS"] = df["CHits"] / df["Years"]
df["AVG_RUN"] = df["HmRun"] + df["CRuns"] / 2 * df["Years"]
df["AVG_CRBI"] = df["CRBI"] / df["Years"]
df["AVG_PUTOUT"] = df["PutOuts"] / df["Years"]
df["RUNS_NOT_BATTED"] = df["RBI"] - df["Runs"]
df["AVG_WALKS"] = df["Walks"] / df["Years"]
df["HMRUN_RATE"] = df["HmRun"] / df["CRuns"]
df["PUTOUT_RATE"] =  df["Assists"]/ df["PutOuts"] 
df["AVG_ATBAT"] = df["AtBat"] / df["Years"]



## DATA PREPROCESSING

In [ ]:
df[num_cols].skew()

In [ ]:
added_cols = ["AVG_HITS","AVG_RUN", "AVG_CRBI", "AVG_PUTOUT", "RUNS_NOT_BATTED", "AVG_WALKS","HMRUN_RATE","PUTOUT_RATE","AVG_ATBAT"] 
df[added_cols].skew()

In [ ]:
for i in df[added_cols]:
    plt.figure() 
    sns.distplot(df[i])

In [ ]:
new_cols = ["AVG_RUN","AVG_PUTOUT", "AVG_WALKS","HMRUN_RATE","AVG_WALKS","PUTOUT_RATE","AVG_ATBAT" ]

In [ ]:
crim_cols = [col for col in num_cols if (col != "Hits" and col != "AtBat" and col != "Salary" and col!="Runs")]

all_cols = crim_cols + new_cols

In [ ]:
crim_log = np.log(df[all_cols])

In [ ]:
# crim_sqrt = np.sqrt(df[all_cols])
crim_sqrt.skew()

In [ ]:
df[all_cols] = np.log(df[all_cols])

### ENCODING

In [ ]:
df = pd.get_dummies(df, columns = ['League', 'Division', 'NewLeague'], drop_first = True)

In [ ]:
df = df.dropna()
df

### MODELING

In [ ]:
df_1 = df.copy()

In [ ]:
y=df_1["Salary"]
X=df_1.drop("Salary", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=42)

In [ ]:
lgb_model = LGBMRegressor().fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:

lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [500, 1000],
               "max_depth": [5, 8, 10],
              "num_leaves":[32,64,128],
              "min_data_in_leaf":[100,1000]}

lgbm_cv_model = GridSearchCV(lgb_model,
                             lgbm_params,
                             cv=10,
                             n_jobs=-1,
                             verbose=2).fit(X_train, y_train)

lgbm_cv_model.best_params_

In [ ]:
 lgbm_tuned = LGBMRegressor(**lgbm_cv_model.best_params_).fit(X_train, y_train)
 y_pred = lgbm_tuned.predict(X_test)
 np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:

Importance = pd.DataFrame({'Importance':lgb_model.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', figsize=(14,12))

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

In [ ]:
xgb = XGBRegressor()
xgb_tuned_params = {
 'max_depth':[5,10],
 'n_estimators': [100,500]
}
#metrics: f1_micro
xgb_tuned_model = GridSearchCV(estimator = xgb, param_grid = xgb_tuned_params, scoring='f1_micro',n_jobs=-1,verbose = 10, cv=5).fit(X_train, y_train)


xgb_tuned_model.best_params_

In [ ]:
xgb_tuned = XGBRegressor(**lgbm_cv_model.best_params_).fit(X_train, y_train)
y_pred = xgb_tuned_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))